# Data Sampling

## Setup

In [1]:
import sys

!{sys.executable} -m pip install --quiet --user --upgrade pandas==1.*
!{sys.executable} -m pip install --quiet --user --upgrade -r requirements.txt

In [2]:
from goodies import *
import pandas as pd
import os

## Data Collection

In [3]:
from dcollect import plugins

modules = {'http': plugins.fasthttp()}
headers = None

### YouTube (United States)

Initial setup. Be sure to have your API key ready. For details on how to obtain an API key, read [YouTube Data API Overview, Introduction: Before you start](https://developers.google.com/youtube/v3/getting-started#before-you-start).

In [4]:
from dcollect import api_youtube as youtube
from dcollect import api_youtubei as youtubei

# This key is for testing ONLY. DO NOT release to the public!
api_experiment = False
api_key_testing = None
api_key = os.environ.get('YOUTUBE_API_KEY') or api_key_testing

if not api_key:
    api_key = os.environ.get('YOUTUBE_EXPLORER_API_KEY')
    if api_key: 
        api_experiment = True
    else: 
        api_key = input('YouTube Data API Key: ')
        api_experiment = (input('Is this an explorer key? [Y/N]: ') == 'Y')

dataset_id = os.environ.get('DATASET_NAME')
if dataset_id == None:
    dataset_id = input('Dataset Name: ')
    
sample_size_per_query_default = 1000000
sample_size_per_query = os.environ.get('SAMPLE_SIZE_PER_QUERY')    
if sample_size_per_query == None:
    sample_size_per_query = input('Sample size per query: ') or sample_size_per_query_default
    
sample_size_per_query = int(sample_size_per_query)

#### Search

##### STEP 1  Data Collection

In [5]:
# create a YouTube API object
youtube_o = youtube.api(
    modules = modules,
    headers = headers,
    key = api_key,
    experiment = api_experiment
)

# create a YouTube Internals API object
youtubei_o = youtubei.api(
    modules = modules,
    headers = headers
)

In [6]:
def df_search_gen(*args, **kwargs):
    from dcollect.utils.log import log
    log.enable(level = log.levels.WARNING)
    import concurrent.futures

    df_search = None
    df_info = None
    df_channels = None
    df_ads = None
    
    def worker_df_search(*args, **kwargs):
        nonlocal df_search
        df_search = df_from_json(
            youtube_o.video.search(
                *args, **kwargs
            )
        )
        
    def worker_df_info():
        nonlocal df_info
        df_info = df_from_json(
            youtube_o.video.info(
                id = df_search['id']
            )
        )
            
    def worker_df_ads():
        nonlocal df_ads
        df_ads = df_from_json(
            youtubei_o.ad.placements(
                id = df_search['id'],
                throttle_size = None
            )
        )
            
    def worker_df_channels():
        nonlocal df_channels
        df_channels = df_from_json(
            youtube_o.channel.info(
                id = df_search['creator.id']
            )
        )
            
    # - search
    worker_df_search(*args, **kwargs)
    
    with concurrent.futures.ThreadPoolExecutor() as executor:
        for worker in worker_df_info, worker_df_ads, worker_df_channels:
            executor.submit(worker)
        executor.shutdown(wait = True)
                
    return df_search, df_info, df_channels, df_ads

In [7]:
def df_search_gen_bulk(paramlist: list):
    import concurrent.futures
    
    results = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(df_search_gen, **param) for param in paramlist]
        results = [f.result() for f in concurrent.futures.as_completed(futures)]
        
    return results

In [8]:
import string

param_default = {
    'count': sample_size_per_query
}

paramlist = []
for c in string.ascii_lowercase:
    param = dict(param_default)
    param.update({
        'keyword': c
    })
    paramlist.append(param)
    
df_search = pd.DataFrame()
df_info = pd.DataFrame()
df_channels = pd.DataFrame()
df_ads = pd.DataFrame()

results = df_search_gen_bulk(paramlist)

In [9]:
def transpose(l):
    return list(map(list, zip(*l)))

df_search_res, df_info_res, df_channels_res, df_ads_res = transpose(results)

df_search = pd.concat(df_search_res, copy = False)
df_info = pd.concat(df_info_res, copy = False)
df_channels = pd.concat(df_channels_res, copy = False)
df_ads = pd.concat(df_ads_res, copy = False)

df_report([df_search, df_info, df_channels, df_ads])

/home/jared/lab/group068_wi21/EDA/goodies.py:57: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df.describe()


---

# Results - 

## Data Preview

,id,title,description,time,tags,creator.id
0,DruYNHN4FYY,Yes Yes Vegetables Song + More Nursery Rhymes ...,Watch your favorite song by clicking a title b...,2018-09-07 07:00:03+00:00,None,UCbCmjCuTUZos6Inko4u57UQ
1,k9-t3uOrhs4,"UNTV: C-NEWS | March 11, 2021","Publiko, pinapayuhang magdoble ingat vs. COVID...",2021-03-11 05:30:04+00:00,None,UC3XaG-7UVi2vD8ZZEMNnnpw
2,npjKVpGBQW8,Still Fresh? Oddly Satisfying Cleaning Out OLD...,Twitter https://www.twitter.com/cookieswirlc/ ...,2020-08-03 19:02:25+00:00,None,UCelMeixAOTs2OQAAi9wU8-g
3,eM_KBmA1aw4,ജന ലക്ഷങ്ങൾ സംബന്ധിക്കുന്ന മദനീയം സ്വലാത്ത് | ...,C Media Is YouTube Channels Introduced Islamic...,2021-03-10 19:07:16+00:00,None,UCJpTA2LgMKJEIgCISqYnRWA
4,KBh_GPoirSQ,Escape The Hospital Roblox Obby,Today in Roblox my cookie fans and I are tryin...,2021-02-27 21:22:16+00:00,None,UCelMeixAOTs2OQAAi9wU8-g


...

## Data Preview

,id,ads
0,LUpnFNUmfKw,None
1,Ruesy-2hihs,None
2,RJFoBfV2w2g,None
3,i6gyjMecDA8,None
4,IC25DUOSIpw,[{'kind': 'AD_PLACEMENT_KIND_COMMAND_TRIGGERED...


## Stats

,id,ads
count,780,644
unique,778,301
top,jtJVAdSJuFQ,"[{'kind': 'AD_PLACEMENT_KIND_START', 'offset':..."
freq,2,71


---

##### STEP 2  Data Cleaning

In [10]:
# - * (filter)
def drop_common(df, df_other, *args, **kwargs):
    return df.drop(columns = df.columns & df_other.columns, *args, **kwargs)

# - search
df_search.set_index(['id'], inplace = True)
# - info
df_info.set_index(['id'], inplace = True)
# - channels
df_channels = df_channels.add_prefix('creator.')
df_channels.set_index(['creator.id'], inplace = True)
# - ads
df_ads.set_index(['id'], inplace = True)

# drop common columns to avoid clashing
# in this case, only `df_search` and `df_info` have merging conflicts
drop_common(df_search, df_info, inplace = True)

<ipython-input-10-776a02e81705>:3: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead
  return df.drop(columns = df.columns & df_other.columns, *args, **kwargs)


In [11]:
# - search (with details)
df_search_details = df_search.copy()
# - info
df_search_details = df_search_details.merge(
    df_info, 
    right_index = True, 
    left_on = 'id', 
    copy = False
)
# - ads
df_search_details = df_search_details.merge(
    df_ads, 
    right_index = True, 
    left_on = 'id', 
    copy = False
)

##### STEP 3  Data Inspection

In [12]:
# take a brief look at our data
df_report(df_search_details, name = 'Search Result')
df_report(df_channels, name = 'Search Result (Channels)')

---

# Results - Search Result

## Data Preview

,title,description,time,length,tags,category,creator.id,stats.like,stats.dislike,stats.comment,stats.view,video.quality,ads
id,,,,,,,,,,,,,
_wjfogsjeyE,"Sao Việt b,ật khóc hay tin d,ữ ""kh.ủng kh.iê'p...","Sao Việt b,ật khóc hay tin d,ữ ""kh.ủng kh.iê'p...",2021-03-08 08:55:11+00:00,0 days 00:02:16,"[sao việt, sao viet, nghe si, nghệ sĩ, showbiz...",People & Blogs,UCjvdDj9VfzQfpWhkwkjbT4g,139.0,18.0,25.0,34964.0,HD,"[{'kind': 'AD_PLACEMENT_KIND_START', 'offset':..."
_wjfogsjeyE,"Sao Việt b,ật khóc hay tin d,ữ ""kh.ủng kh.iê'p...","Sao Việt b,ật khóc hay tin d,ữ ""kh.ủng kh.iê'p...",2021-03-08 08:55:11+00:00,0 days 00:02:16,"[sao việt, sao viet, nghe si, nghệ sĩ, showbiz...",People & Blogs,UCjvdDj9VfzQfpWhkwkjbT4g,139.0,18.0,25.0,34960.0,HD,"[{'kind': 'AD_PLACEMENT_KIND_START', 'offset':..."
_wjfogsjeyE,"Sao Việt b,ật khóc hay tin d,ữ ""kh.ủng kh.iê'p...","Sao Việt b,ật khóc hay tin d,ữ ""kh.ủng kh.iê'p...",2021-03-08 08:55:11+00:00,0 days 00:02:16,"[sao việt, sao viet, nghe si, nghệ sĩ, showbiz...",People & Blogs,UCjvdDj9VfzQfpWhkwkjbT4g,139.0,18.0,25.0,34960.0,HD,"[{'kind': 'AD_PLACEMENT_KIND_START', 'offset':..."
_wjfogsjeyE,"Sao Việt b,ật khóc hay tin d,ữ ""kh.ủng kh.iê'p...","Sao Việt b,ật khóc hay tin d,ữ ""kh.ủng kh.iê'p...",2021-03-08 08:55:11+00:00,0 days 00:02:16,"[sao việt, sao viet, nghe si, nghệ sĩ, showbiz...",People & Blogs,UCjvdDj9VfzQfpWhkwkjbT4g,139.0,18.0,25.0,34964.0,HD,"[{'kind': 'AD_PLACEMENT_KIND_START', 'offset':..."
_wjfogsjeyE,"Sao Việt b,ật khóc hay tin d,ữ ""kh.ủng kh.iê'p...","Sao Việt b,ật khóc hay tin d,ữ ""kh.ủng kh.iê'p...",2021-03-08 08:55:11+00:00,0 days 00:02:16,"[sao việt, sao viet, nghe si, nghệ sĩ, showbiz...",People & Blogs,UCjvdDj9VfzQfpWhkwkjbT4g,139.0,18.0,25.0,34960.0,HD,"[{'kind': 'AD_PLACEMENT_KIND_START', 'offset':..."


## Stats

...

## Data Preview

,creator.title,creator.description,creator.time,creator.stats.follower,creator.stats.view,creator.stats.post
creator.id,,,,,,
UCgFFMzzYIYYn3T6M-v8Kfsw,Double Bubble,Привет ты на Double Bubble!\nМеня зовут Аня ;)...,2017-12-18 07:59:38+00:00,2370000.0,998774746,619
UC6yEA7_U_Q4aY9TAwKFLb3g,Alrima Officiel,Bienvenue sur la chaîne officielle d'Alrima,2012-10-28 19:37:08+00:00,1020000.0,249319555,51
UCMtcfSnNybg3uhiljDCYQrA,Mr. Benz,Hey! I am Mr.Benz & I am sharing my passion re...,2017-01-16 19:02:09+00:00,105000.0,25515543,58
UCJpTA2LgMKJEIgCISqYnRWA,Cmedia Live,🔴 C Media Is YouTube Channels Introduced Islam...,2016-10-19 17:56:06+00:00,388000.0,41280092,1848
UC4_AL-HBcYBjCTyNnMw0DOw,자동차읽어주는남자,자동차 읽어주는 남자\n- 자동차 강의문의 및 차량지원 문의\n- 이메일 : lsh...,2012-07-15 15:39:59+00:00,66300.0,26489632,475


## Stats

,creator.stats.follower,creator.stats.view,creator.stats.post
count,3.923000e+03,4.383000e+03,4383.000000
mean,4.885402e+06,2.555694e+09,3482.732603
std,1.694917e+07,1.256854e+10,16946.381008
min,4.100000e+01,0.000000e+00,1.000000
25%,8.750000e+04,1.740609e+07,66.000000
50%,5.870000e+05,1.478008e+08,284.000000
75%,2.770000e+06,9.987747e+08,1175.000000
max,1.750000e+08,1.462732e+11,291948.000000


---

##### STEP 4  Data Archiving

In [13]:
pickle_proto = 3

dataset = eda_utils.dataset(f'dsamples/youtube_search_{dataset_id}.dataset')
dataset.update('youtube_search.pkl', df_search, proto = pickle_proto)
dataset.update('youtube_search_info.pkl', df_ads, proto = pickle_proto)
dataset.update('youtube_search_ads.pkl', df_ads, proto = pickle_proto)
dataset.update('youtube_search_channels.pkl', df_channels, proto = pickle_proto)
dataset.update('youtube_search_details.pkl', df_search_details, proto = pickle_proto)

# verify that we saved the correct data
df_report(dataset.load('youtube_search_details.pkl'), name = 'Search Result (Verification)')
df_report(dataset.load('youtube_search_channels.pkl'), name = 'Search Result (Channels) (Verification)')

---

# Results - Search Result (Verification)

## Data Preview

,title,description,time,length,tags,category,creator.id,stats.like,stats.dislike,stats.comment,stats.view,video.quality,ads
id,,,,,,,,,,,,,
_wjfogsjeyE,"Sao Việt b,ật khóc hay tin d,ữ ""kh.ủng kh.iê'p...","Sao Việt b,ật khóc hay tin d,ữ ""kh.ủng kh.iê'p...",2021-03-08 08:55:11+00:00,0 days 00:02:16,"[sao việt, sao viet, nghe si, nghệ sĩ, showbiz...",People & Blogs,UCjvdDj9VfzQfpWhkwkjbT4g,139.0,18.0,25.0,34964.0,HD,"[{'kind': 'AD_PLACEMENT_KIND_START', 'offset':..."
_wjfogsjeyE,"Sao Việt b,ật khóc hay tin d,ữ ""kh.ủng kh.iê'p...","Sao Việt b,ật khóc hay tin d,ữ ""kh.ủng kh.iê'p...",2021-03-08 08:55:11+00:00,0 days 00:02:16,"[sao việt, sao viet, nghe si, nghệ sĩ, showbiz...",People & Blogs,UCjvdDj9VfzQfpWhkwkjbT4g,139.0,18.0,25.0,34960.0,HD,"[{'kind': 'AD_PLACEMENT_KIND_START', 'offset':..."
_wjfogsjeyE,"Sao Việt b,ật khóc hay tin d,ữ ""kh.ủng kh.iê'p...","Sao Việt b,ật khóc hay tin d,ữ ""kh.ủng kh.iê'p...",2021-03-08 08:55:11+00:00,0 days 00:02:16,"[sao việt, sao viet, nghe si, nghệ sĩ, showbiz...",People & Blogs,UCjvdDj9VfzQfpWhkwkjbT4g,139.0,18.0,25.0,34960.0,HD,"[{'kind': 'AD_PLACEMENT_KIND_START', 'offset':..."
_wjfogsjeyE,"Sao Việt b,ật khóc hay tin d,ữ ""kh.ủng kh.iê'p...","Sao Việt b,ật khóc hay tin d,ữ ""kh.ủng kh.iê'p...",2021-03-08 08:55:11+00:00,0 days 00:02:16,"[sao việt, sao viet, nghe si, nghệ sĩ, showbiz...",People & Blogs,UCjvdDj9VfzQfpWhkwkjbT4g,139.0,18.0,25.0,34964.0,HD,"[{'kind': 'AD_PLACEMENT_KIND_START', 'offset':..."
_wjfogsjeyE,"Sao Việt b,ật khóc hay tin d,ữ ""kh.ủng kh.iê'p...","Sao Việt b,ật khóc hay tin d,ữ ""kh.ủng kh.iê'p...",2021-03-08 08:55:11+00:00,0 days 00:02:16,"[sao việt, sao viet, nghe si, nghệ sĩ, showbiz...",People & Blogs,UCjvdDj9VfzQfpWhkwkjbT4g,139.0,18.0,25.0,34960.0,HD,"[{'kind': 'AD_PLACEMENT_KIND_START', 'offset':..."


## Stats

...

## Data Preview

,creator.title,creator.description,creator.time,creator.stats.follower,creator.stats.view,creator.stats.post
creator.id,,,,,,
UCgFFMzzYIYYn3T6M-v8Kfsw,Double Bubble,Привет ты на Double Bubble!\nМеня зовут Аня ;)...,2017-12-18 07:59:38+00:00,2370000.0,998774746,619
UC6yEA7_U_Q4aY9TAwKFLb3g,Alrima Officiel,Bienvenue sur la chaîne officielle d'Alrima,2012-10-28 19:37:08+00:00,1020000.0,249319555,51
UCMtcfSnNybg3uhiljDCYQrA,Mr. Benz,Hey! I am Mr.Benz & I am sharing my passion re...,2017-01-16 19:02:09+00:00,105000.0,25515543,58
UCJpTA2LgMKJEIgCISqYnRWA,Cmedia Live,🔴 C Media Is YouTube Channels Introduced Islam...,2016-10-19 17:56:06+00:00,388000.0,41280092,1848
UC4_AL-HBcYBjCTyNnMw0DOw,자동차읽어주는남자,자동차 읽어주는 남자\n- 자동차 강의문의 및 차량지원 문의\n- 이메일 : lsh...,2012-07-15 15:39:59+00:00,66300.0,26489632,475


## Stats

,creator.stats.follower,creator.stats.view,creator.stats.post
count,3.923000e+03,4.383000e+03,4383.000000
mean,4.885402e+06,2.555694e+09,3482.732603
std,1.694917e+07,1.256854e+10,16946.381008
min,4.100000e+01,0.000000e+00,1.000000
25%,8.750000e+04,1.740609e+07,66.000000
50%,5.870000e+05,1.478008e+08,284.000000
75%,2.770000e+06,9.987747e+08,1175.000000
max,1.750000e+08,1.462732e+11,291948.000000


---